In [19]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, normalization):
        super(CNN, self).__init__()
        self.a = False
        if(normalization == nn.GroupNorm):
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
            self.n1 = normalization(4, 32)

            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.n2 = normalization(8, 64)

            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
            self.n3 = normalization(16, 128)

            self.pool = nn.MaxPool2d(2, 2)

            self.fc1 = nn.Linear(128 * 16 * 16, 128) 
            self.fc2 = nn.Linear(128, 10)
        elif(normalization == nn.LayerNorm):
            self.a = True
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
            
            self.pool = nn.MaxPool2d(2, 2)

            self.fc1 = nn.Linear(128 * 16 * 16, 128) 
            self.n1 = normalization(128)
            self.fc2 = nn.Linear(128, 10)
        else:
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
            self.n1 = normalization(32)

            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.n2 = normalization(64)

            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
            self.n3 = normalization(128)

            self.pool = nn.MaxPool2d(2, 2)

            self.fc1 = nn.Linear(128 * 16 * 16, 128) 
            self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        if self.a == True:
            x = F.relu(self.conv1(x))
        
            x = F.relu(self.conv2(x))

            x = F.relu(self.conv3(x))
            
            x = self.pool(x)
            
            x = x.view(-1, 128 * 16 * 16)  

            x = F.relu(self.n1(self.fc1(x)))

            x = self.fc2(x)
        else:
            x = F.relu(self.n1(self.conv1(x)))
            
            x = F.relu(self.n2(self.conv2(x)))

            x = F.relu(self.n3(self.conv3(x)))
            
            x = self.pool(x)
            
            x = x.view(-1, 128 * 16 * 16)  

            x = F.relu(self.fc1(x))
            x = self.fc2(x)

        return x


In [16]:
# train
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 数据加载和预处理
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

train_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=True, download=True, transform=transforms_train),
    batch_size=128, shuffle=True)

def train(normalization, name):

    model = CNN(normalization)
    optimizer = optim.Adamax(model.parameters(), lr=0.01)


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    # 训练过程
    for epoch in range(1, 30):
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.CrossEntropyLoss()(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    # 保存模型
    torch.save(model.state_dict(), f'./model/{name}_cnn.pt')


transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

test_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=False, download=True, transform=transforms_train),
    batch_size=1000, shuffle=True)

def test(normalization, name):
    model = CNN(normalization)
    model.load_state_dict(torch.load(f'./model/{name}_cnn.pt'))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: name:{name} Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.4f}%)\n')


Files already downloaded and verified
Files already downloaded and verified


In [20]:
train(nn.BatchNorm2d, "Batch")
test(nn.BatchNorm2d, "Batch")

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.369488
Train Epoch: 1 [1280/50000 (3%)]	Loss: 21.686745
Train Epoch: 1 [2560/50000 (5%)]	Loss: 4.020526
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.305258
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.308775
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.302589
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.302893
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.301783
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.304905
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.303416
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.305804
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.297814
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.308616
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.294218
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.301779
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.300224
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.304179
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.304296
Train Epoch: 1 [23040/50000 (46%)]	Loss: 2.307471
Train Epoch: 1 [24320/50000 (49%)]	Loss: 2.308244
Train Epoch: 1

In [ ]:
train(nn.LayerNorm, "Layer")
test(nn.LayerNorm, "Layer")

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.409150
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.229070
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.127481
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.147497
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.976982
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.143390
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.943477
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.970143
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.825297
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.676309
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.733504
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.677279
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.687470
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.707554
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.643952
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.523214
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.605085
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.592322
Train Epoch: 1 [23040/50000 (46%)]	Loss: 1.357984
Train Epoch: 1 [24320/50000 (49%)]	Loss: 1.578236
Train Epoch: 1 

In [ ]:

train(nn.InstanceNorm2d, "Instance")
test(nn.InstanceNorm2d, "Instance")

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.377289
Train Epoch: 1 [1280/50000 (3%)]	Loss: 40.450844
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.343574
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.298513
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.303223
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.302945
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.305356
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.304612
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.304615
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.301339
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.302348
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.304358
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.305161
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.302415
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.300228
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.304722
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.301206
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.301105
Train Epoch: 1 [23040/50000 (46%)]	Loss: 2.306799
Train Epoch: 1 [24320/50000 (49%)]	Loss: 2.297677
Train Epoch: 1

In [ ]:
train(nn.GroupNorm, "Group")
test(nn.GroupNorm, "Group")

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.302163
Train Epoch: 1 [1280/50000 (3%)]	Loss: 7.296113
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.278037
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.265427
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.220511
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.257658
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.262649
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.241428
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.273082
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.230642
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.206458
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.273513
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.217037
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.199029
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.082695
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.233122
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.216006
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.205095
Train Epoch: 1 [23040/50000 (46%)]	Loss: 2.227217
Train Epoch: 1 [24320/50000 (49%)]	Loss: 2.275910
Train Epoch: 1 